# Meta-analysis with METAL

METAT is a command line tool, takes in a script that documenting the input sumstat files, the field of each of those sumstat input file, and then the end of scipt.

METAL analysis is enssentially a weighted sum of Z score, therefore the input of snps from each chrm can be processed seperately. To make sure the same region was analysis, the input of this workflow shall be a list of path to the sumstat, join by " " to be analyzed at once.

Also, The scheme of weighting at the moment is set to be standard error, changing it to sample size would require the upstream input have a columns named sample size, which they dont have it yet.

At the moment, the AVERAGEFREQ, MINMAXFREQ, and GENOMICCONTROL are by default set to off, turning on each of the option require additional input from upstream file.

The output file name of the seconde step will by default contains a 1 linking the desinateted prefix and surfix, which lead to the f'{wd}/{name}1.METAL.txt' file name

In [ ]:
[global]
parameter: sumstat_list = list
parameter: name = str
parameter: variant_id = 'variant_id'
parameter: Sample_size = 'N'
parameter: ALLELE = 'alt ref'
parameter: FREQ   = 'EFFECT_ALLELE_FREQ'
parameter: Beta = 'beta'
parameter: SE = 'se'
parameter: PVAL   = 'pval'
parameter: wd = path
parameter: SCHEME = 'STDERR'
parameter: AVERAGEFREQ = "OFF"
parameter: MINMAXFREQ  = "OFF"
parameter: GENOMICCONTROL  = "OFF"
import pandas as pd

sumstat_inv = pd.read_csv(sumstat_list, sep = "\t").values.tolist()





### Step.1 Script generation
The METAL script contains three sections, the first is the analysis parameters, the second is the inputs files, and the third is the ending of script.

In [ ]:
[METAL_1,generate_METAL_script]
input:  sumstat_inv, group_by = 1
output: METAL_script = f'{wd}/{name}.{_input[0]}.METAL_script.txt'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        echo '
        # Meta-analysis weighted by:
        SCHEME   $[SCHEME]
        
        # Whether do genomics control
        GENOMICCONTROL $[GENOMICCONTROL]
        
        # Whether do AVERAGEFREQ or MINMAXFREQ control
        AVERAGEFREQ $[AVERAGEFREQ]
        MINMAXFREQ $[MINMAXFREQ] 
        OUTFILE $[wd]/$[name] .METAL.txt' > $[_output]

        for i in $[" ".join(_input[1:len(_input)])] ; do 
        echo "     
        MARKER   $[variant_id]
        WEIGHT   $[Sample_size]
        ALLELE   $[ALLELE]
        #FREQ     $[FREQ]
        EFFECT   $[Beta]
        STDERR   $[SE]
        PVAL     $[PVAL]
        PROCESS  $i " >> $[_output];
        done 
        
        echo '
        ANALYZE' >> $[_output]

## Step 2. Metal analysis

In [ ]:
[METAL_2,Excute_METAL_script]
input: group_with = "sumstat_inv"
output: METAL_result = f'{wd}/{name}.{_sumstat_inv[0]}.1.METAL.txt'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
       metal $[_input]

## Step 3. Reformat the output so that they are in the same formmat as the non metaled one

In [ ]:
[METAL_3,Output_reformatting]
input: group_with = "sumstat_inv"
output: METAL_result = f'{wd}/{name}.{_sumstat_inv[0]}.METAL.txt'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
        import pandas as pd
        df = pd.read_csv($[_input:r],sep = "\t")
        df.columns = ["variant_id","alt","ref","Weight","beta","pval","Direction"]
        df = df.assign(se = 1)
        df.to_csv("$[_output]",sep = "\t",index = 0)

In [ ]:
[METAL_4,recipe]
input: group_by = "all"
output: METAL_result = f'{wd}/{name}.METAL_list.txt'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        import pandas as pd
        df = pd.read_csv(sumstat_list, sep = "\t")["#chr"].assign( "METAL_dir"  = [$[_input:r,]] )
        df.to_csv("$[_output]",sep = "\t",index = 0)